In [43]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [44]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

/content/gdrive/My Drive/IDLSProject-main


In [45]:
!pwd

/content/gdrive/My Drive/IDLSProject-main


In [46]:
import numpy as np

In [47]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000, 1])


In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 5000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]

In [49]:
# write to file.
filename = "cnn-lstm-imdb.csv"
def write_to_csv(epochs, train_loss, val_loss, val_acc, time_train):
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename)
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(file_name)
    df_csv['Test_Accuracy']  = accuracy
    df_metrics.to_csv(filename)

In [50]:
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        #self.cnn = CNN(vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len)
        self.lstm = nn.LSTM(
            input_size=3*filter_size, 
            hidden_size=hidden_size, 
            num_layers=1,
            batch_first=True)
        self.dense = nn.Linear(hidden_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x):
        
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        # print("x1")
        # print(x1.shape)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        # print("f1")
        # print(f1.shape)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        c_out = torch.cat([f1,f2,f3],dim=1)
        r_in = c_out.view(batch_size,1, -1)
        lstm_out, _ = self.lstm(r_in)
        out = self.dense(lstm_out[:, -1, :])
        return self.sig(out)

In [52]:
import time
def train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device):
    train_loss = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    start = time.time()
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.data.item()
        train_loss.append(total_loss / len(train_dl))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(loss.item())
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(correct/total)
            val_loss_epoch.append(np.mean(val_loss))
            end = time.time() - start
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(end)
    write_to_csv(epochs, train_loss, val_loss_epoch, val_accuracy_epoch, time_train)
    return model

def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    append_to_csv(epochs, correct/total)
    print("Accuracy:", correct/total)

In [54]:
import torch.optim as optim
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Combine(vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()
train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device)

Epoch: 0, BCELoss: 0.6934087495009105
Val Loss: 0.690 	Val Acc: 0.507
Epoch: 1, BCELoss: 0.47231336623430253
Val Loss: 0.378 	Val Acc: 0.832
Epoch: 2, BCELoss: 0.33977820302049316
Val Loss: 0.336 	Val Acc: 0.855
Epoch: 3, BCELoss: 0.29638115329047043
Val Loss: 0.326 	Val Acc: 0.863
Epoch: 4, BCELoss: 0.26937139260272186
Val Loss: 0.314 	Val Acc: 0.866
Epoch: 5, BCELoss: 0.2466618535295129
Val Loss: 0.308 	Val Acc: 0.867
Epoch: 6, BCELoss: 0.23352375840147335
Val Loss: 0.312 	Val Acc: 0.865
Epoch: 7, BCELoss: 0.2175015158827106
Val Loss: 0.301 	Val Acc: 0.874
Epoch: 8, BCELoss: 0.2060503106129666
Val Loss: 0.303 	Val Acc: 0.873
Epoch: 9, BCELoss: 0.19473556411763032
Val Loss: 0.311 	Val Acc: 0.872


Combine(
  (embedding): Embedding(5000, 32)
  (conv1): Conv2d(1, 100, kernel_size=(3, 32), stride=(1, 1))
  (conv2): Conv2d(1, 100, kernel_size=(4, 32), stride=(1, 1))
  (conv3): Conv2d(1, 100, kernel_size=(5, 32), stride=(1, 1))
  (mp1): MaxPool1d(kernel_size=498, stride=498, padding=0, dilation=1, ceil_mode=False)
  (mp2): MaxPool1d(kernel_size=497, stride=497, padding=0, dilation=1, ceil_mode=False)
  (mp3): MaxPool1d(kernel_size=496, stride=496, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(300, 100, batch_first=True)
  (dense): Linear(in_features=100, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [58]:
torch.save(model.state_dict(),"./cnn-lstm-imdb.pth")

In [59]:
def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    print("Accuracy:", correct/total)

In [ ]:
test(model, test_dl, epochs)